<a href="https://colab.research.google.com/github/lingyu001/Recommendation/blob/master/YoutubeDNN_breakdown.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
! wget http://files.grouplens.org/datasets/movielens/ml-1m.zip -O ./ml-1m.zip
! wget https://raw.githubusercontent.com/lingyu001/Recommendation/master/preprocess.py -O preprocess.py
! wget https://raw.githubusercontent.com/lingyu001/Recommendation/master/model_utils.py -O model_utils.py
! unzip -o ml-1m.zip 

--2022-10-12 20:25:14--  http://files.grouplens.org/datasets/movielens/ml-1m.zip
Resolving files.grouplens.org (files.grouplens.org)... 128.101.65.152
Connecting to files.grouplens.org (files.grouplens.org)|128.101.65.152|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 5917549 (5.6M) [application/zip]
Saving to: ‘./ml-1m.zip’

./ml-1m.zip         100%[===================>]   5.64M  31.6MB/s    in 0.2s    

2022-10-12 20:25:14 (31.6 MB/s) - ‘./ml-1m.zip’ saved [5917549/5917549]

--2022-10-12 20:25:15--  https://raw.githubusercontent.com/lingyu001/Recommendation/master/preprocess.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 6687 (6.5K) [text/plain]
Saving to: ‘preprocess.py’

preprocess.py       100%[===================>]   6.53K

In [2]:
!pip install ipython-autotime
%load_ext autotime

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
time: 1.23 ms (started: 2022-10-12 20:25:22 +00:00)


In [3]:
! pip install -q deepmatch

time: 6.29 s (started: 2022-10-12 20:25:22 +00:00)


In [4]:
import pandas as pd
from preprocess import gen_data_set, gen_model_input
from sklearn.preprocessing import LabelEncoder
from tensorflow.python.keras import backend as K
from tensorflow.python.keras.models import Model
from model_utils import DNN, EmbeddingIndex, NoMask, PoolingLayer, l2_normalize,\
 reduce_mean, SampledSoftmaxLayer, get_item_embedding, sampledsoftmaxloss

time: 8.08 s (started: 2022-10-12 20:25:28 +00:00)


In [5]:
%load_ext autoreload
%autoreload 2

time: 104 ms (started: 2022-10-12 20:25:36 +00:00)


# Data

In [6]:
data_path = "./"

unames = ['user_id','gender','age','occupation','zip']
user = pd.read_csv(data_path+'ml-1m/users.dat',sep='::',header=None,names=unames)


time: 291 ms (started: 2022-10-12 20:25:36 +00:00)


/usr/local/lib/python3.7/dist-packages/pandas/util/_decorators.py:311: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  return func(*args, **kwargs)


In [7]:
user = user.sample(50, random_state=2022)

time: 96 ms (started: 2022-10-12 20:25:36 +00:00)


In [8]:
rnames = ['user_id','movie_id','rating','timestamp']
ratings = pd.read_csv(data_path+'ml-1m/ratings.dat',sep='::',header=None,names=rnames)
mnames = ['movie_id','title','genres']
movies = pd.read_csv(data_path+'ml-1m/movies.dat',sep='::',header=None,names=mnames,encoding="unicode_escape")
movies['genres'] = list(map(lambda x: x.split('|')[0], movies['genres'].values))

data = pd.merge(pd.merge(ratings,movies),user)#.iloc[:10000] # taking 1w
print(data.shape)
print(data.user_id.nunique())
print(data.movie_id.nunique())
data.head(2)

(8767, 10)
50
2088


,user_id,movie_id,rating,timestamp,title,genres,gender,age,occupation,zip
0,1204,1193,4,974843566,One Flew Over the Cuckoo's Nest (1975),Drama,M,25,7,95076
1,1204,661,3,974846590,James and the Giant Peach (1996),Animation,M,25,7,95076


time: 12.1 s (started: 2022-10-12 20:25:37 +00:00)


# Feature Engineering

In [16]:
sparse_features = ["movie_id", "user_id",
                    "gender", "age", "occupation", "zip", "genres"]

SEQ_LEN = 50
NUMBER_NEG_SAMPLE = 1 # used for generate 1:1 positive:negative samples


time: 46 ms (started: 2022-10-12 20:28:09 +00:00)


In [17]:
feature_max_idx = {}
for feature in sparse_features:
    lbe = LabelEncoder()
    data[feature] = lbe.fit_transform(data[feature]) + 1 # add one to all the encoded categories labels
    feature_max_idx[feature] = data[feature].max() + 1

time: 104 ms (started: 2022-10-12 20:28:09 +00:00)


In [18]:
feature_max_idx

{'movie_id': 2089,
 'user_id': 51,
 'gender': 3,
 'age': 7,
 'occupation': 14,
 'zip': 49,
 'genres': 19}

time: 71.8 ms (started: 2022-10-12 20:28:09 +00:00)


In [19]:
data.head(2)

,user_id,movie_id,rating,timestamp,title,genres,gender,age,occupation,zip
0,8,610,4,974843566,One Flew Over the Cuckoo's Nest (1975),8,2,2,8,43
1,8,368,3,974846590,James and the Giant Peach (1996),3,2,2,8,43


time: 58.2 ms (started: 2022-10-12 20:28:10 +00:00)


In [20]:
user_profile = data[["user_id", "gender", "age", "occupation", "zip"]].drop_duplicates('user_id')
user_profile.set_index("user_id", inplace=True)
item_profile = data[["movie_id"]].drop_duplicates('movie_id')
user_profile.shape, item_profile.shape

((50, 4), (2088, 1))

time: 76.3 ms (started: 2022-10-12 20:28:10 +00:00)


In [21]:
user_item_list = data.groupby("user_id")['movie_id'].apply(list)
user_item_list.head()

user_id
1    [613, 468, 1590, 1520, 1526, 1253, 679, 575, 8...
2    [1832, 1275, 624, 1917, 826, 1558, 274, 1510, ...
3    [679, 575, 148, 1080, 366, 1588, 624, 1917, 66...
4    [307, 1558, 17, 574, 209, 1838, 763, 712, 593,...
5    [1275, 613, 1531, 347, 1590, 1520, 1463, 1526,...
Name: movie_id, dtype: object

time: 55.5 ms (started: 2022-10-12 20:28:10 +00:00)


In [22]:
train_set, test_set = gen_data_set(data, SEQ_LEN, NUMBER_NEG_SAMPLE)

100%|██████████| 50/50 [00:00<00:00, 326.50it/s]

8 8
time: 242 ms (started: 2022-10-12 20:28:10 +00:00)


In [23]:
# reviewerID,  - user id
# pos_list[i], - movie id
# 1, - label y
# hist[::-1][:seq_len], - most recent watched movie id list before watching this movie, up to 50
# seq_len
# genres_hist[::-1][:seq_len], - most recent watched movie's genres list before watching this movie, up to 50
# genres_list[i], - this movie's genre
# rating_list[i] - this movie's rating
# train_set[12]

time: 54.4 ms (started: 2022-10-12 20:28:10 +00:00)


In [24]:
train_model_input, train_label = gen_model_input(train_set, user_profile, SEQ_LEN)
test_model_input, test_label = gen_model_input(test_set, user_profile, SEQ_LEN)
train_label.shape, train_model_input['user_id'].shape, test_model_input['user_id'].shape, test_label.shape

/content/preprocess.py:86: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  train_seq_genres = np.array([line[5] for line in train_set])


((17334,), (17334,), (50,), (50,))

time: 386 ms (started: 2022-10-12 20:28:10 +00:00)


# Create Embeddings

In [25]:
from collections import Counter
train_counter = Counter(train_model_input['movie_id'])
item_count = [train_counter.get(i,0) for i in range(feature_max_idx['movie_id'])]
len(train_counter), type(train_counter), len(item_count)

(2088, collections.Counter, 2089)

time: 75.2 ms (started: 2022-10-12 20:28:11 +00:00)


In [26]:
import tensorflow as tf
from tensorflow.python.keras.initializers import RandomNormal, Zeros
from tensorflow.python.keras.layers import Embedding, Input, Lambda
from tensorflow.python.keras.regularizers import l2
if tf.__version__ >= '2.0.0':
    tf.compat.v1.disable_eager_execution()
else:
    K.set_learning_phase(True)

time: 54 ms (started: 2022-10-12 20:28:11 +00:00)


### Create Feature Embeddings



In [27]:
# item features
item_feature_columns = ['movie_id']
# User features
user_feature_columns = ['user_id',
                        "gender",
                        "age",
                        "occupation",
                        "zip",
                        'hist_movie_id',
                        'hist_genres',
                        'hist_len'
                        ]


time: 57.2 ms (started: 2022-10-12 20:28:11 +00:00)


In [28]:
feature_max_idx

{'movie_id': 2089,
 'user_id': 51,
 'gender': 3,
 'age': 7,
 'occupation': 14,
 'zip': 49,
 'genres': 19}

time: 62.5 ms (started: 2022-10-12 20:28:11 +00:00)


In [29]:
user_sparse_feature_columns = user_feature_columns[0:5]
user_sparse_feature_columns

['user_id', 'gender', 'age', 'occupation', 'zip']

time: 60.2 ms (started: 2022-10-12 20:28:11 +00:00)


In [30]:
user_seq_sparse_feature_columns = user_feature_columns[5:-1]
user_seq_sparse_feature_columns 

['hist_movie_id', 'hist_genres']

time: 52.9 ms (started: 2022-10-12 20:28:11 +00:00)


In [31]:
# embedding input and output specification
embeddings_initializer = RandomNormal(mean=0.0, stddev=0.0001, seed=2020)
embedding_dim = 32
l2_reg = 1e-6
# use a dict to collect embeddings
sparse_embedding = {}

Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor


time: 60.7 ms (started: 2022-10-12 20:28:11 +00:00)


In [32]:
# user embeddings
for name in user_sparse_feature_columns: 
  emb = Embedding(
      input_dim = feature_max_idx[name], 
      output_dim = 16,
      embeddings_initializer=embeddings_initializer,
      embeddings_regularizer=l2(l2_reg),
      name='sparse_' + 'emb_' + name
  )
  emb.trainable = True
  sparse_embedding[name] = emb
for name in user_seq_sparse_feature_columns: 
   emb = Embedding(feature_max_idx[name[5:]], embedding_dim,
                            embeddings_initializer=embeddings_initializer,
                            embeddings_regularizer=l2(
                                l2_reg),
                            name='sparse_' + 'seq_emb_'  + name,
                            mask_zero=False) # True
   emb.trainable = True
   sparse_embedding[name] = emb

time: 59.3 ms (started: 2022-10-12 20:28:11 +00:00)


In [33]:
# item embeddings
item_feature_name = item_feature_columns[0] # only include item id 
emb = Embedding(
      input_dim = feature_max_idx[item_feature_name], 
      output_dim = embedding_dim,
      embeddings_initializer=embeddings_initializer,
      embeddings_regularizer=l2(l2_reg),
      name='sparse_' + 'emb_' + item_feature_name
  )
sparse_embedding[item_feature_name] = emb

time: 50 ms (started: 2022-10-12 20:28:11 +00:00)


In [34]:
sparse_embedding # include both user and item embedding layers

{'user_id': <tensorflow.python.keras.layers.embeddings.Embedding at 0x7f87c54a9310>,
 'gender': <tensorflow.python.keras.layers.embeddings.Embedding at 0x7f87c52caad0>,
 'age': <tensorflow.python.keras.layers.embeddings.Embedding at 0x7f87c5489c50>,
 'occupation': <tensorflow.python.keras.layers.embeddings.Embedding at 0x7f87c5489e90>,
 'zip': <tensorflow.python.keras.layers.embeddings.Embedding at 0x7f87c54df910>,
 'hist_movie_id': <tensorflow.python.keras.layers.embeddings.Embedding at 0x7f87c54a9e90>,
 'hist_genres': <tensorflow.python.keras.layers.embeddings.Embedding at 0x7f87c54876d0>,
 'movie_id': <tensorflow.python.keras.layers.embeddings.Embedding at 0x7f87c54831d0>}

time: 58 ms (started: 2022-10-12 20:28:11 +00:00)


# Create User Inputs


## Create Inputs Tensor

In [35]:
# user features inputs
from collections import OrderedDict
user_features = OrderedDict()
for name in user_sparse_feature_columns:
  user_features[name] = Input(
                shape=(1,), 
                name=name, 
                dtype="int32"
                )
for name in user_seq_sparse_feature_columns:
  user_features[name] = Input(
                shape=(SEQ_LEN,), 
                name=name, 
                dtype="int32"
                )
length_name = 'hist_len'
if length_name in user_feature_columns:
  user_features[length_name] = Input((1,), name=length_name, dtype='int32')

time: 82.9 ms (started: 2022-10-12 20:28:11 +00:00)


In [36]:
user_inputs_list = list(user_features.values())
user_inputs_list

[<tf.Tensor 'user_id:0' shape=(None, 1) dtype=int32>,
 <tf.Tensor 'gender:0' shape=(None, 1) dtype=int32>,
 <tf.Tensor 'age:0' shape=(None, 1) dtype=int32>,
 <tf.Tensor 'occupation:0' shape=(None, 1) dtype=int32>,
 <tf.Tensor 'zip:0' shape=(None, 1) dtype=int32>,
 <tf.Tensor 'hist_movie_id:0' shape=(None, 50) dtype=int32>,
 <tf.Tensor 'hist_genres:0' shape=(None, 50) dtype=int32>,
 <tf.Tensor 'hist_len:0' shape=(None, 1) dtype=int32>]

time: 67.7 ms (started: 2022-10-12 20:28:11 +00:00)


In [37]:
# item feature inputs
item_features = OrderedDict()
for name in item_feature_columns:
  item_features[name] = Input(
                shape=(1,), 
                name=name, 
                dtype="int32"
                )
item_inputs_list = list(item_features.values())


time: 58.7 ms (started: 2022-10-12 20:28:11 +00:00)


In [38]:
item_inputs_list

[<tf.Tensor 'movie_id:0' shape=(None, 1) dtype=int32>]

time: 57.5 ms (started: 2022-10-12 20:28:11 +00:00)


In [39]:
item_inputs_list[0]

<tf.Tensor 'movie_id:0' shape=(None, 1) dtype=int32>

time: 74.7 ms (started: 2022-10-12 20:28:12 +00:00)


## Embed Inputs

In [40]:
# Embed sparse inputs
from collections import defaultdict
user_sparse_embeded_input = defaultdict(list)

for name in user_sparse_feature_columns:
  user_sparse_embeded_input[name] = sparse_embedding[name](user_features[name])

time: 113 ms (started: 2022-10-12 20:28:12 +00:00)


In [41]:
# Embed the sequence sparse inputs, need to add pooling layer to it
seq_embeded_dict = defaultdict(list)

for name in user_seq_sparse_feature_columns:
  seq_embeded_dict[name] = sparse_embedding[name](user_features[name])

# from deepctr.layers.sequence import SequencePoolingLayer
# for name in user_seq_sparse_feature_columns:
#   user_seq_sparse_embeded_input[name] = SequencePoolingLayer('mean', supports_masking=False)([seq_embeded_dict[name], user_features[length_name]])

from tensorflow.keras.layers import AveragePooling1D
user_seq_sparse_embeded_input = defaultdict(list)
for name in user_seq_sparse_feature_columns:
  user_seq_sparse_embeded_input[name] = AveragePooling1D(pool_size=50, padding='valid')(seq_embeded_dict[name])

time: 93.8 ms (started: 2022-10-12 20:28:12 +00:00)


In [42]:
# vec = AveragePooling1D(pool_size=50, strides = 50, padding='valid')
# vec(seq_embeded_dict[name])

time: 59.2 ms (started: 2022-10-12 20:28:12 +00:00)


In [43]:
# from deepctr.layers.sequence import SequencePoolingLayer
# vec2 = SequencePoolingLayer('mean', supports_masking=False)
# vec2([seq_embeded_dict[name], 35])

time: 55.4 ms (started: 2022-10-12 20:28:12 +00:00)


In [44]:
# combine embeded inputs into a list
user_sparse_embedding_list = list(user_sparse_embeded_input.values()) + list(user_seq_sparse_embeded_input.values())
user_sparse_embedding_list

[<tf.Tensor 'sparse_emb_user_id/embedding_lookup/Identity_1:0' shape=(None, 1, 16) dtype=float32>,
 <tf.Tensor 'sparse_emb_gender/embedding_lookup/Identity_1:0' shape=(None, 1, 16) dtype=float32>,
 <tf.Tensor 'sparse_emb_age/embedding_lookup/Identity_1:0' shape=(None, 1, 16) dtype=float32>,
 <tf.Tensor 'sparse_emb_occupation/embedding_lookup/Identity_1:0' shape=(None, 1, 16) dtype=float32>,
 <tf.Tensor 'sparse_emb_zip/embedding_lookup/Identity_1:0' shape=(None, 1, 16) dtype=float32>,
 <tf.Tensor 'average_pooling1d/Squeeze:0' shape=(None, 1, 32) dtype=float32>,
 <tf.Tensor 'average_pooling1d_1/Squeeze:0' shape=(None, 1, 32) dtype=float32>]

time: 61 ms (started: 2022-10-12 20:28:12 +00:00)


## Combine Embeded Input (User only)

In [45]:
from tensorflow.python.keras.layers import Flatten, Concatenate, Layer, Add

time: 44.1 ms (started: 2022-10-12 20:28:12 +00:00)


In [46]:
user_dnn_input = Flatten()(Concatenate(axis=2)(user_sparse_embedding_list))
user_dnn_input

<tf.Tensor 'flatten/Reshape:0' shape=(None, 144) dtype=float32>

time: 70.1 ms (started: 2022-10-12 20:28:12 +00:00)


# Sepecify Model Layers

### User Layers

In [47]:
from tensorflow.keras import layers

time: 55.8 ms (started: 2022-10-12 20:28:12 +00:00)


In [48]:
user_dnn_hidden_units = (128,64, embedding_dim)
dnn_activation='relu'
dnn_use_bn=False
l2_reg_dnn=0
l2_reg_embedding=1e-6
dnn_dropout=0
output_activation='linear'
temperature=0.05,
# sampler_config=sampler_config
seed=1024

time: 57.5 ms (started: 2022-10-12 20:28:12 +00:00)


In [49]:
from deepctr.layers import DNN

time: 65.3 ms (started: 2022-10-12 20:28:12 +00:00)


In [50]:
dnn_layers = DNN(hidden_units = user_dnn_hidden_units, 
                  activation='relu', 
                  l2_reg=0, 
                  dropout_rate=0, 
                  use_bn=False, 
                  output_activation='linear',
                  seed=1024
                  )



time: 72.5 ms (started: 2022-10-12 20:28:12 +00:00)


In [51]:
user_dnn_output = dnn_layers(user_dnn_input)
user_dnn_output = l2_normalize(user_dnn_output)
user_dnn_output

<tf.Tensor 'lambda/l2_normalize:0' shape=(None, 32) dtype=float32>

time: 676 ms (started: 2022-10-12 20:28:13 +00:00)


### Item Layer

In [52]:
# Create item index tensor
item_vocabulary_size = feature_max_idx[item_feature_name]
item_index = EmbeddingIndex(list(range(item_vocabulary_size)))(item_features[item_feature_name])

# embed the item index input
item_embeding = sparse_embedding[item_feature_name]
item_embedding_weight = NoMask()(item_embeding(item_index))
pooling_item_embedding_weight = PoolingLayer()([item_embedding_weight])
pooling_item_embedding_weight = l2_normalize(pooling_item_embedding_weight)
pooling_item_embedding_weight


<tf.Tensor 'lambda_1/l2_normalize:0' shape=(2089, 32) dtype=float32>

time: 303 ms (started: 2022-10-12 20:28:13 +00:00)


In [53]:
data[item_feature_name].min(), data[item_feature_name].max()

(1, 2088)

time: 73.7 ms (started: 2022-10-12 20:28:14 +00:00)


In [54]:
item_vocabulary_size, item_index.eval(session=K.get_session()).shape

(2089, (2089,))

time: 139 ms (started: 2022-10-12 20:28:14 +00:00)


In [55]:
item_embedding_weight.eval(session=K.get_session()).shape

(2089, 32)

time: 62.5 ms (started: 2022-10-12 20:28:14 +00:00)


In [56]:
pooling_item_embedding_weight.eval(session=K.get_session()).shape

(2089, 32)

time: 75.4 ms (started: 2022-10-12 20:28:14 +00:00)


In [57]:
# .eval(session=K.get_session())

time: 62 ms (started: 2022-10-12 20:28:14 +00:00)


### Output Layer

In [58]:
output = SampledSoftmaxLayer()(
    [pooling_item_embedding_weight, user_dnn_output, item_features[item_feature_name]])
output

<tf.Tensor 'sampled_softmax_layer/ExpandDims:0' shape=(None, 1) dtype=float32>

time: 242 ms (started: 2022-10-12 20:28:14 +00:00)


In [59]:
# from model_utils import SampledSoftmaxLayer
# sf = SampledSoftmaxLayer()

time: 56 ms (started: 2022-10-12 20:28:14 +00:00)


In [60]:
# from tensorflow.keras.layers import Softmax
# sf2 = Softmax()
# import numpy as np
# x = np.asarray([[1., 2., 1.]])
# x
# sf2(x).eval(session=K.get_session())

time: 54.1 ms (started: 2022-10-12 20:28:14 +00:00)


In [61]:
pooling_item_embedding_weight

<tf.Tensor 'lambda_1/l2_normalize:0' shape=(2089, 32) dtype=float32>

time: 60.4 ms (started: 2022-10-12 20:28:14 +00:00)


In [62]:
user_dnn_output

<tf.Tensor 'lambda/l2_normalize:0' shape=(None, 32) dtype=float32>

time: 64.3 ms (started: 2022-10-12 20:28:15 +00:00)


In [63]:
item_features[item_feature_name]

<tf.Tensor 'movie_id:0' shape=(None, 1) dtype=int32>

time: 69.3 ms (started: 2022-10-12 20:28:15 +00:00)


time: 79.8 ms (started: 2022-10-12 20:28:15 +00:00)


## Build Model

In [64]:
from tensorflow.python.keras.models import Model

time: 63.9 ms (started: 2022-10-12 20:28:15 +00:00)


In [65]:
model = Model(inputs=user_inputs_list + item_inputs_list, outputs=output)

model.__setattr__("user_input", user_inputs_list)
model.__setattr__("user_embedding", user_dnn_output)

model.__setattr__("item_input", item_inputs_list)
model.__setattr__("item_embedding",
                  get_item_embedding(pooling_item_embedding_weight, item_features[item_feature_name]))


time: 67.3 ms (started: 2022-10-12 20:28:15 +00:00)


# Model Compile and Training

In [66]:
model.compile(optimizer="adam", loss=sampledsoftmaxloss)

time: 224 ms (started: 2022-10-12 20:28:15 +00:00)


In [67]:
history = model.fit(train_model_input, train_label,
                    batch_size=512, epochs=100, verbose=1, validation_split=0.0, )

Train on 17334 samples
Epoch 1/100
17334/17334 [==============================] - 2s 87us/sample - loss: 5.1611
Epoch 2/100
17334/17334 [==============================] - 1s 58us/sample - loss: 5.1005
Epoch 3/100
17334/17334 [==============================] - 1s 56us/sample - loss: 5.0940
Epoch 4/100
17334/17334 [==============================] - 1s 56us/sample - loss: 5.0504
Epoch 5/100
17334/17334 [==============================] - 1s 57us/sample - loss: 5.0441
Epoch 6/100
17334/17334 [==============================] - 1s 56us/sample - loss: 5.0430
Epoch 7/100
17334/17334 [==============================] - 1s 58us/sample - loss: 5.0044
Epoch 8/100
17334/17334 [==============================] - 1s 57us/sample - loss: 4.9707
Epoch 9/100
17334/17334 [==============================] - 1s 55us/sample - loss: 4.9404
Epoch 10/100
17334/17334 [==============================] - 1s 53us/sample - loss: 4.9038
Epoch 11/100
17334/17334 [==============================] - 1s 56us/sample - loss: 4.9

In [68]:
res = model.predict(train_model_input)

/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/engine/training.py:2458: UserWarning: `Model.state_updates` will be removed in a future version. This property should not be used in TensorFlow 2.0, as `updates` are applied automatically.
  warnings.warn('`Model.state_updates` will be removed in a future version. '


time: 2.88 s (started: 2022-10-12 20:29:49 +00:00)


# Predict: Generate Item and User Embeddings

In [69]:
# 4. Generate user features for testing and full item features for retrieval
test_user_model_input = test_model_input
all_item_model_input = {"movie_id": item_profile['movie_id'].values,}

user_embedding_model = Model(inputs=model.user_input, outputs=model.user_embedding)
item_embedding_model = Model(inputs=model.item_input, outputs=model.item_embedding)

user_embs = user_embedding_model.predict(test_user_model_input, batch_size=2 ** 12)
# user_embs = user_embs[:, i, :]  # i in [0,k_max) if MIND
item_embs = item_embedding_model.predict(all_item_model_input, batch_size=2 ** 12)

print(user_embs.shape)
print(item_embs.shape)

(50, 32)
(2088, 32)
time: 231 ms (started: 2022-10-12 20:29:51 +00:00)


In [70]:
item_profile['movie_id'].values.min(), item_profile['movie_id'].values.max()

(1, 2088)

time: 58.8 ms (started: 2022-10-12 20:29:52 +00:00)


In [71]:
item_embs.shape

(2088, 32)

time: 57.1 ms (started: 2022-10-12 20:29:52 +00:00)


In [72]:
user_embs.shape, item_embs.shape

((50, 32), (2088, 32))

time: 55.4 ms (started: 2022-10-12 20:29:52 +00:00)


In [73]:
prob_matrix = user_embs.dot(item_embs.T)

time: 54.8 ms (started: 2022-10-12 20:29:52 +00:00)


In [74]:
prob_matrix.shape

(50, 2088)

time: 75.4 ms (started: 2022-10-12 20:29:52 +00:00)


# Recommendation

In [75]:
! pip install faiss-cpu

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
time: 3.53 s (started: 2022-10-12 20:29:52 +00:00)


In [76]:
import numpy as np
import faiss
from tqdm import tqdm
from deepmatch.utils import recall_N

time: 95.1 ms (started: 2022-10-12 20:29:56 +00:00)


In [77]:
test_true_label = {line[0]:[line[1]] for line in test_set}

import numpy as np
import faiss
from tqdm import tqdm
from deepmatch.utils import recall_N

index = faiss.IndexFlatIP(embedding_dim)
# faiss.normalize_L2(item_embs)
index.add(item_embs)
# faiss.normalize_L2(user_embs)
D, I = index.search(np.ascontiguousarray(user_embs), 50)
s = []
hit = 0
pred_label = {}
for i, uid in tqdm(enumerate(test_user_model_input['user_id'])):
    try:
        pred = [item_profile['movie_id'].values[x] for x in I[i]]
        filter_item = None
        recall_score = recall_N(test_true_label[uid], pred, N=50)
        s.append(recall_score)
        pred_label[uid] = pred
        if test_true_label[uid] in pred:
            hit += 1
    except:
        print(i)
print("")
print("recall", np.mean(s))
print("hit rate", hit / len(test_user_model_input['user_id']))

50it [00:00, 1776.49it/s]


recall 0.1
hit rate 0.1
time: 103 ms (started: 2022-10-12 20:29:56 +00:00)


In [78]:
test_user_model_input['user_id']

array([11, 29, 13, 23, 10, 27, 39, 24, 45, 21, 31, 41, 43,  2, 37, 34, 20,
       38, 15,  5, 48, 19,  7, 47, 35, 36, 46, 26, 50, 49,  1, 28, 44, 14,
       16,  4, 42, 18, 25, 22, 33,  6, 17, 32, 40, 12,  8, 30,  9,  3])

time: 203 ms (started: 2022-10-12 20:29:56 +00:00)


# Validation

In [79]:
df_train = pd.DataFrame(train_set)

time: 200 ms (started: 2022-10-12 20:29:56 +00:00)


In [80]:
df_train.head()

,0,1,2,3,4,5,6,7
0,5,1134,1,"[1393, 549, 816, 1417, 993, 1471, 674, 1633, 1...",50,"[5, 5, 5, 5, 5, 3, 5, 5, 5, 5, 3, 8, 8, 8, 8, ...",5,4.0
1,1,568,0,"[1305, 674, 1821, 1288, 569, 1179, 1108, 1361,...",50,"[5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 2, 3, 5, ...",8,NaN
2,37,1261,0,"[496, 194, 1272, 1028, 1369, 710, 172, 618, 13...",50,"[8, 8, 5, 5, 8, 8, 8, 8, 8, 8, 1, 8, 4, 8, 8, ...",1,NaN
3,7,344,1,"[1613, 762, 1053, 251, 878, 1305, 1590, 324, 1...",50,"[8, 8, 11, 5, 5, 5, 5, 4, 5, 5, 8, 8, 1, 5, 8,...",1,4.0
4,9,1995,1,"[329, 838, 1139, 610, 1944, 516, 1013, 411, 11...",50,"[1, 5, 8, 8, 1, 4, 12, 5, 8, 5, 1, 5, 1, 2, 2,...",8,4.0


time: 104 ms (started: 2022-10-12 20:29:56 +00:00)


In [90]:
test_user_model_input['user_id']

array([11, 29, 13, 23, 10, 27, 39, 24, 45, 21, 31, 41, 43,  2, 37, 34, 20,
       38, 15,  5, 48, 19,  7, 47, 35, 36, 46, 26, 50, 49,  1, 28, 44, 14,
       16,  4, 42, 18, 25, 22, 33,  6, 17, 32, 40, 12,  8, 30,  9,  3])

time: 135 ms (started: 2022-10-12 20:36:05 +00:00)


In [91]:
user_idx = 0
uid = 11

time: 75.6 ms (started: 2022-10-12 20:36:14 +00:00)


### Score match

In [92]:
D[user_idx]

array([0.7900038 , 0.77942777, 0.7558832 , 0.73660195, 0.7346535 ,
       0.71701527, 0.7098384 , 0.7079486 , 0.70121956, 0.6957664 ,
       0.6937254 , 0.69180423, 0.68864036, 0.6866999 , 0.68196756,
       0.67856765, 0.6651815 , 0.6648223 , 0.658706  , 0.65683186,
       0.65341485, 0.6526418 , 0.65239036, 0.6515457 , 0.6513677 ,
       0.6462875 , 0.6453591 , 0.6451587 , 0.6441928 , 0.6437572 ,
       0.6430403 , 0.6410727 , 0.6383361 , 0.63721925, 0.6245674 ,
       0.62239975, 0.61863315, 0.60851276, 0.608477  , 0.6084454 ,
       0.60816985, 0.60681784, 0.601196  , 0.6006198 , 0.6004869 ,
       0.598561  , 0.59778094, 0.59627056, 0.5957043 , 0.5948411 ],
      dtype=float32)

time: 52.6 ms (started: 2022-10-12 20:36:17 +00:00)


In [93]:
sorted_index = prob_matrix.argsort()[:,-50:]
-np.sort(-prob_matrix[user_idx][sorted_index[user_idx]])

array([0.7900038 , 0.77942777, 0.7558832 , 0.7366019 , 0.7346535 ,
       0.71701527, 0.7098384 , 0.7079486 , 0.70121956, 0.6957664 ,
       0.6937254 , 0.69180423, 0.68864036, 0.6866999 , 0.68196756,
       0.67856765, 0.6651815 , 0.6648223 , 0.658706  , 0.65683186,
       0.65341485, 0.6526418 , 0.65239036, 0.6515457 , 0.6513677 ,
       0.6462875 , 0.6453591 , 0.6451587 , 0.6441929 , 0.6437572 ,
       0.6430403 , 0.6410727 , 0.6383361 , 0.6372192 , 0.6245674 ,
       0.62239975, 0.61863315, 0.60851276, 0.608477  , 0.6084454 ,
       0.60816985, 0.60681784, 0.601196  , 0.6006198 , 0.6004869 ,
       0.598561  , 0.59778094, 0.59627056, 0.5957043 , 0.5948411 ],
      dtype=float32)

time: 59 ms (started: 2022-10-12 20:36:18 +00:00)


### Index match

In [94]:
np.array([item_profile['movie_id'].values[x] for x in I[user_idx]])

array([1016, 1616, 1142, 1833,  692,  952,  471, 1603,  438,  619,  941,
       1163,  458, 1109, 1087,   43, 1843,  660,  319, 1105, 1753,  627,
        486, 1812,  634, 1963,    4,  489, 1652,  148, 1625,  602,  881,
        561,  866,  282,  299,  614,  623, 1986,  474,  616,  152,  461,
        612,  629, 1608, 2069, 1941,  320])

time: 49 ms (started: 2022-10-12 20:36:21 +00:00)


In [95]:
I[user_idx]

array([ 201,  797, 1671, 1049,  449, 1614, 1297, 1777,   67,  391,  252,
       1548,   40, 1227,  129, 1462, 1280,  705,   71,  185,  300,  230,
       1829, 1069,  282,  730, 1945,  938, 1689,  156, 1370,  867, 1239,
       1358,  243, 1923,  183,  194,  558, 1623,  718,  389,   50, 1285,
         17,  430,  549, 1094, 1051, 1651])

time: 48.7 ms (started: 2022-10-12 20:36:22 +00:00)


In [96]:
(sorted_index[user_idx])[::-1]

array([ 201,  797, 1671, 1049,  449, 1614, 1297, 1777,   67,  391,  252,
       1548,   40, 1227,  129, 1462, 1280,  705,   71,  185,  300,  230,
       1829, 1069,  282,  730, 1945,  938, 1689,  156, 1370,  867, 1239,
       1358,  243, 1923,  183,  194,  558, 1623,  718,  389,   50, 1285,
         17,  430,  549, 1094, 1051, 1651])

time: 53.3 ms (started: 2022-10-12 20:36:23 +00:00)


In [100]:
res_idx_len = []
res_org_len = []
for i, uid in tqdm(enumerate(test_user_model_input['user_id'])):
  res_idx_len.append(len(set(df_train[df_train[0]==uid][1].unique()) & set(sorted_index[user_idx])))
  res_org_len.append(len(set(df_train[df_train[0]==uid][1].unique()) & set([item_profile['movie_id'].values[x] for x in I[user_idx]])))



50it [00:00, 520.39it/s]

time: 152 ms (started: 2022-10-12 20:40:39 +00:00)


In [103]:
pd.Series(res_idx_len).describe(), pd.Series(res_org_len).describe()

(count    50.000000
 mean      7.780000
 std       9.205788
 min       0.000000
 25%       2.000000
 50%       3.500000
 75%      12.000000
 max      31.000000
 dtype: float64, count    50.000000
 mean     10.040000
 std       8.973794
 min       0.000000
 25%       3.000000
 50%       7.000000
 75%      15.000000
 max      32.000000
 dtype: float64)

time: 230 ms (started: 2022-10-12 20:41:18 +00:00)


In [97]:
len(set(df_train[df_train[0]==uid][1].unique()) & set(sorted_index[user_idx]))

0

time: 53 ms (started: 2022-10-12 20:36:27 +00:00)


In [98]:
len(set(df_train[df_train[0]==uid][1].unique()) & set([item_profile['movie_id'].values[x] for x in I[user_idx]]))

15

time: 60.3 ms (started: 2022-10-12 20:36:29 +00:00)


## Reference: Logic of embedding index match to original label

In [89]:
from keras.layers import Embedding, Input
from keras.models import Sequential, Model

inputs = Input(shape=(1,))
embedding = Embedding(5, 3, input_length=1, name='cat_col')(inputs)
model = Model(inputs, embedding)

x = np.array([0,1,2,3,4]).reshape(5,1)
# x = np.array([1,3,4,2,0]).reshape(5,1)

# labels = np.zeros((5,1,3))

print (model.predict(x))
print (model.get_layer('cat_col').get_weights()[0])

assert np.array_equal(model.predict(x).reshape(-1), model.get_layer('cat_col').get_weights()[0].reshape(-1))

[[[ 0.03897731  0.03492305 -0.03595017]]

 [[ 0.04051191  0.04440126 -0.02900038]]

 [[-0.01612986 -0.04406906  0.01908295]]

 [[ 0.00931183 -0.02604643 -0.00010148]]

 [[ 0.03751112  0.02173838 -0.01694157]]]
[[ 0.03897731  0.03492305 -0.03595017]
 [ 0.04051191  0.04440126 -0.02900038]
 [-0.01612986 -0.04406906  0.01908295]
 [ 0.00931183 -0.02604643 -0.00010148]
 [ 0.03751112  0.02173838 -0.01694157]]
time: 255 ms (started: 2022-10-12 20:29:57 +00:00)


/usr/local/lib/python3.7/dist-packages/keras/engine/training_v1.py:2079: UserWarning: `Model.state_updates` will be removed in a future version. This property should not be used in TensorFlow 2.0, as `updates` are applied automatically.
  updates=self.state_updates,
